In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

In [ ]:
def clean_entity_names(df):
    # Standardize company/supplier names
    df['Company_Name'] = df['Company_Name'].apply(standardize_company_name)

    # Standardize sponsoring government agency names
    df['Government_Agency'] = df['Government_Agency'].apply(standardize_government_agency)

    return df

In [ ]:
def standardize_company_name(name):
    # Implement logic for standardizing company names (e.g., removing variations)
    # You can use string matching or other techniques
    # Example: Replace variations of Tata Steel with a common name "Tata Steel"
    if "Tata Steel" in name:
        return "Tata Steel"

In [ ]:
def standardize_government_agency(name):
    # Implement logic for standardizing government agency names
    # Example: Replace variations of "Department of Defense" with "DoD"
    if "Department of Defense" in name:
        return "DoD"

In [ ]:
# Read data
data = pd.read_csv('sam_data.csv')  # Replace 'sam_data.csv' with actual file name

In [ ]:
# Clean data using the defined function
cleaned_data = clean_entity_names(data)

In [ ]:
# Save cleaned data to a new CSV file
cleaned_data.to_csv('cleaned_sam_data.csv', index=False)

In [ ]:
from fuzzywuzzy import fuzz, process

In [ ]:
# Function for fuzzy string matching
def fuzzy_match(target, choices):
    match, score = process.extractOne(target, choices, scorer=fuzz.token_sort_ratio)
    if score >= 80:  # Adjust threshold based on your data
        return match
    else:
        return target

In [ ]:
# Read data
data = pd.read_csv('sample_data.csv')  # Replace 'sample_data.csv' with actual file name

In [ ]:
# Fuzzy match and standardize company names
data['Company_Name'] = data['Company_Name'].apply(lambda x: fuzzy_match(x, data['Company_Name'].unique()))

In [ ]:
# Fuzzy match and standardize government agency names
data['Government_Agency'] = data['Government_Agency'].apply(lambda x: fuzzy_match(x, data['Government_Agency'].unique()))

In [ ]:
# Save cleaned data to a new CSV file
data.to_csv('automated_cleaned_data.csv', index=False)

In [ ]:
import schedule
import time

In [ ]:
# Function for automated cleaning
def automate_cleaning():
    data = pd.read_csv('new_data.csv')  # Replace 'new_data.csv' with actual file name
    data['Company_Name'] = data['Company_Name'].apply(lambda x: fuzzy_match(x, data['Company_Name'].unique()))
    data['Government_Agency'] = data['Government_Agency'].apply(lambda x: fuzzy_match(x, data['Government_Agency'].unique()))
    data.to_csv('automated_cleaned_data.csv', index=False)

In [ ]:
# Schedule script to run every day at midnight
schedule.every().day.at("00:00").do(automate_cleaning)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)